In [19]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [8]:
df = pd.read_csv('C:/Users/rishi/Desktop/ML/Datasets/Algerian_forest_fires_cleaned_dataset.csv')
df.head()

,day,month,year,Temperature,RH,Ws,Rain,FFMC,DMC,DC,ISI,BUI,FWI,Classes,Region
0,1,6,2012,29,57,18,0.0,65.7,3.4,7.6,1.3,3.4,0.5,not fire,0
1,2,6,2012,29,61,13,1.3,64.4,4.1,7.6,1.0,3.9,0.4,not fire,0
2,3,6,2012,26,82,22,13.1,47.1,2.5,7.1,0.3,2.7,0.1,not fire,0
3,4,6,2012,25,89,13,2.5,28.6,1.3,6.9,0.0,1.7,0.0,not fire,0
4,5,6,2012,27,77,16,0.0,64.8,3.0,14.2,1.2,3.9,0.5,not fire,0


In [9]:
df.drop(['day', 'month', 'year'], axis = 1, inplace = True)
df.Classes = np.where(df.Classes.str.contains('not fire'), 0, 1)

In [10]:
X = df.drop('FWI', axis = 1)
y = df.FWI

In [15]:
from sklearn.model_selection import train_test_split as tts

X_train, X_test, y_train, y_test = tts(X, y, test_size = 0.33, random_state = 10)

In [16]:
def correlation(dataset, threshold) :
    col_corr = set()
    corr_matrix = dataset.corr()

    for i in range(len(corr_matrix.columns)) :
        for j in range(i) :
            if abs(corr_matrix.iloc[i, j])  > threshold :
                colname = corr_matrix.columns[i]
                col_corr.add(colname)

    return col_corr


corr_features = correlation(X_train, 0.85)

In [17]:
X_train.drop(corr_features, axis = 1, inplace = True)
X_test.drop(corr_features, axis = 1, inplace = True)

In [18]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [30]:
from sklearn.linear_model import Lasso

regressor = Lasso()
regressor.fit(X_train, y_train)
y_test_prediction = regressor.predict(X_test)

print("Slope or Coefficient:", regressor.coef_)
print("Intercept:", regressor.intercept_)

Slope or Coefficient: [ 0.         -0.          0.         -0.          0.          2.6976982
  3.81931139  0.         -0.        ]
Intercept: 6.845061728395062


In [31]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

mse = mean_squared_error(y_test, y_test_prediction) # Actual data and Predicted Data
mae = mean_absolute_error(y_test, y_test_prediction)
rmse = np.sqrt(mse)

print("Mean Squared Error:", mse)
print("Mean Absolute Error:", mae)
print("Root Mean Squared Error:", rmse)

Mean Squared Error: 3.1121411377499
Mean Absolute Error: 1.319994782574915
Root Mean Squared Error: 1.7641261683195735


In [32]:
from sklearn.metrics import r2_score

score = r2_score(y_test, y_test_prediction)
print("Accuracy of r squared:", score)

n = (len(y_test) - 1) # No of observations
k = X_test.shape[1] # No of predictor variables
adjusted_score = 1 - ((1 - score) * (n - 1)  / (n - k - 1))
print("Accuracy of Adjusted r squared:", adjusted_score)

Accuracy of r squared: 0.9489776696889856
Accuracy of Adjusted r squared: 0.9424176557918551


## Lasso Cross Validation

In [33]:
from sklearn.linear_model import LassoCV

lassoCV = LassoCV(cv = 5)
lassoCV.fit(X_train, y_train)
y_test_prediction = lassoCV.predict(X_test)

mse = mean_squared_error(y_test, y_test_prediction)
print("Mean Squared Error:", mse)

score = r2_score(y_test, y_test_prediction)
print("Accuracy of r squared:", score)


Mean Squared Error: 0.7935150754099709
Accuracy of r squared: 0.98699063233565
